# Notebook: Analyse Language Diversity

Dieses Notebook analysiert fünf Splits und berechnet Statistiken zur Sprache.

## Packages

In [1]:
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import spacy
import nltk
import json

In [2]:
from spacy.lang.de.stop_words import STOP_WORDS
import string

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nlp = spacy.load("de_core_news_sm")

## Settings

In [5]:
N_SYNTH = 500
LABELS_FIXED = True
N_REAL = 0
MODEL = "Llama13B" # "Llama70B", "GPT-3" 

## Parameters

## Code

In [6]:
def count_tokens(texts):
    token_counts = [] 
    for text in texts:
        tokens = word_tokenize(text)
        token_counts.append(len(tokens))
    return token_counts

In [7]:
def count_unique_lemmas(texts):
    unique_lemmas = set()
    for text in texts:
        doc = nlp(text)
        for token in doc:
            unique_lemmas.add(token.lemma_)
    return len(unique_lemmas)

In [8]:
def remove_stopwords_and_punctuation(text):
    doc = nlp(text)
    cleaned_tokens = [token.lemma_ for token in doc if token.text.lower() not in STOP_WORDS and token.text not in string.punctuation]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

def count_top_n_lemmas(texts, n):
    lemma_counts = {}
    for text in texts:
        print(text)
        cleaned_text = remove_stopwords_and_punctuation(text)
        doc = nlp(cleaned_text)
        for token in doc:
            lemma = token.lemma_
            if lemma in lemma_counts:
                lemma_counts[lemma] += 1
            else:
                lemma_counts[lemma] = 1
    
    sorted_lemmas = sorted(lemma_counts, key=lambda lemma: lemma_counts[lemma], reverse=True)
    top_n_lemmas = sorted_lemmas[:n]
    
    return ', '.join(top_n_lemmas)

In [18]:
total_texts = []
unique_lemma_counts = []
texts_token_counts = []
n_aspects_total = []
n_implicit_aspects_total = []
n_explicit_aspects_total = []

for i in range(0, 5):
    examples_in_split = []
    
    # Load Real Split
    real_path = f"../03 dataset split/real/real_{i}.json"
    with open(real_path, 'r') as file:
        real_data = json.load(file)[:N_REAL]
        examples_in_split = real_data
    
    # Load Synth Split
    fake_path = f"../04 llm synthesis/synth/{MODEL}/{'fixed' if LABELS_FIXED else 'random'}/split_{i}.json"
    with open(fake_path, 'r') as file:
        fake_data = json.load(file)[:N_SYNTH]
        examples_in_split = fake_data
    
    texts = [example["text"] for example in fake_data]
    
    
    # Calculate n tokens in texts
    texts_token_count = count_tokens(texts)
    for count in texts_token_count:
        texts_token_counts.append(count)
    
    # Calcuate unique lemmas in text
    unique_lemma_count = count_unique_lemmas(texts)
    unique_lemma_counts.append(unique_lemma_count)
    
    # Calculate number of aspects (implicit+explicit)
    n_aspects = len([tag["text"] for example in fake_data for tag in example["tags"]])
    n_aspects_total.append(n_aspects)
    
    
    # Calculate number of implicit aspects
    n_implicit_aspects = len([tag["text"] for example in fake_data for tag in example["tags"] if tag["text"] is None])
    n_implicit_aspects_total.append(n_implicit_aspects)
    
    # Calculate number of unique aspect terms
    explicit_aspects = [tag["text"] for example in fake_data for tag in example["tags"] if tag["text"] is not None]
    n_unique_aspect_terms = len(set(explicit_aspects))
    n_explicit_aspects_total.append(n_unique_aspect_terms)
    
    
    # Add to total text collection
    total_texts.extend(texts)

top_n_lemmas = count_top_n_lemmas(total_texts, 5)
unique_lemmas_avg = np.mean(unique_lemma_counts)
texts_token_counts_avg = np.mean(texts_token_counts)
n_aspects_avg = np.mean(n_aspects_total)
n_implicit_aspects_avg = np.mean(n_implicit_aspects_total) / (np.mean(n_implicit_aspects_total) + np.mean(n_explicit_aspects_total))
n_explicit_aspects_avg = np.mean(n_explicit_aspects_total) / (np.mean(n_implicit_aspects_total) + np.mean(n_explicit_aspects_total))

Die Speisen waren ok, und das Ambiente auch.
Im Service und Preis waren wir schon fast nicht überrascht.
Für mich war es auch kein guter Preis - dafür aber ein Gemütlichkeit und ein Service auf dem Niveau, auf das ich mich auch nicht mehr eingestellt hätte.
Hausbackshase war nicht so lecker wie an Fechter.
Preise und Service war eher mittelmäßig, das Essen hingegen super.
Die Speisen waren ok, und das Ambiente auch.
Im Service und Preis waren wir schon fast nicht überrascht.
Für mich war es auch kein guter Preis - dafür aber ein Gemütlichkeit und ein Service auf dem Niveau, auf das ich mich auch nicht mehr eingestellt hätte.
Hausbackshase war nicht so lecker wie an Fechter.
Preise und Service war eher mittelmäßig, das Essen hingegen super.
Die Speisen waren ok, und das Ambiente auch.
Im Service und Preis waren wir schon fast nicht überrascht.
Für mich war es auch kein guter Preis - dafür aber ein Gemütlichkeit und ein Service auf dem Niveau, auf das ich mich auch nicht mehr eingestellt

In [19]:
top_n_lemmas

'Service, Preis, Speise, ok, Ambiente'

In [20]:
unique_lemmas_avg

43.0

In [21]:
texts_token_counts_avg

14.4

In [22]:
n_aspects_avg

13.0

In [23]:
n_implicit_aspects_avg

0.25

In [24]:
n_explicit_aspects_avg

0.75

In [25]:
n_implicit_aspects_avg

0.25